In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import GradientBoostingRegressor
import numpy as np
import pandas as pd
from math import sqrt
from sklearn.metrics import mean_absolute_error as mae
import pickle
from sklearn import metrics
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_predict
import matplotlib.pyplot as plt
#from neupy.algorithms import RBFKMeans
#from neupy.algorithms import GRNN
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import r2_score
from sklearn.svm import SVR
from sklearn import svm
from pyGRNN import GRNN
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn import linear_model
import seaborn as sn
import joblib
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from skopt import BayesSearchCV
import warnings
import heapq
from sklearn.model_selection import RepeatedKFold
from scipy.stats import randint as sp_randint
from skopt.space import Real, Categorical, Integer

warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=DeprecationWarning)

In [ ]:
# Utility function to report best scores+timings
def report_perf(optimizer, X, y, title):
    optimizer.fit(X, y)
    print(title, "best CV score:", optimizer.best_score_)

In [ ]:

'''
Multilayer perceptron (MLP)
'''
from sklearn.multioutput import MultiOutputRegressor

file_name = 'NiTi_PhaseDatesets.xlsx'
dataset = pd.read_excel(file_name, header=0)

#print(dataset)
x1 = dataset['P(W)'].values
x2 = dataset['V(mm/s)'].values
x3 = dataset['h(um)'].values
# x4 = dataset['t(um)'].values
x5 = dataset['LED(J/mm)'].values
# x6 = dataset['SED(J/mm2)'].values
x7 = dataset['VED(J/mm3)'].values
x8 = dataset['Ni'].values
# x9 = dataset['Ti'].values
x10 = dataset['Dmin'].values
x11 = dataset['Dmax'].values
x12 = dataset['D10'].values
# x13 = dataset['D50'].values
x14 = dataset['D90'].values


y1 = dataset['Ms'].values
y2 = dataset['Mf'].values
y3 = dataset['As'].values
y4 = dataset['Af'].values
X = np.column_stack((x1.reshape(-1, 1), 
                     x2.reshape(-1, 1), 
                     x3.reshape(-1, 1),
                #      x4.reshape(-1, 1), 
                     x5.reshape(-1, 1),
                #      x6.reshape(-1, 1),
                     x7.reshape(-1, 1),
                     x8.reshape(-1, 1),
                #      x9.reshape(-1, 1),
                     x10.reshape(-1, 1),
                     x11.reshape(-1, 1),
                     x12.reshape(-1, 1),
                #      x13.reshape(-1, 1),
                     x14.reshape(-1, 1),
                
                     ))

Y = np.column_stack((y1,y2, y3, y4))


MaxGlobal = 0
MaxGlobal_Score = 0
MaxGlobal_Score_train = 0
Max_local = 0



MLP_scores = {'CV': [], 'R2': [],  'R2_train': []}
Accuracy_MLP = []
MLP_RMSE = []
for j in range(50):
    
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state=j)
    xscaler = preprocessing.StandardScaler()
    X_train_standard = xscaler.fit_transform(X_train)
    X_test_standard = xscaler.transform(X_test)

    yscaler = preprocessing.StandardScaler()
    Y_train_standard = yscaler.fit_transform(Y_train)
    
    Y_test_standard = yscaler.transform(Y_test)
    
    mlp = MLPRegressor()
#     wapper = MultiOutputRegressor(mlp)
    params = {
        'hidden_layer_sizes':[(1,4), (2,2,4), (1,2,4)], 
        'activation':['logistic', 'tanh', 'relu'],
        'max_iter':[5000],    
        'solver':['lbfgs'],    
        'learning_rate':['constant'],  
        'random_state':[5]
    }

    total_iter = 20
    #define partitioning
    skf = RepeatedKFold(n_splits=5,random_state=22)
    for baseEstimator in ['GP','RF','ET']:
            opt = BayesSearchCV(estimator = mlp,
            search_spaces=params,
            n_iter=total_iter,n_jobs=-1,cv=skf,return_train_score=False,optimizer_kwargs={'base_estimator': baseEstimator})
            report_perf(opt, X_train_standard, Y_train_standard,'BayesSearchCV_'+baseEstimator)

    opt.fit(X_train_standard,  Y_train_standard)
    best_model = opt.best_estimator_
    
    Cross_validation_MLP = cross_val_score(best_model, X_train_standard, Y_train_standard, cv=5)
    Cross_validation_Accuracy_MLP = Cross_validation_MLP.mean().round(2)
    Accuracy_MLP.append(Cross_validation_Accuracy_MLP)
    Y_pred_MLP = best_model.predict(X_test_standard)
    MLP_test_RMSE = sqrt(np.mean((Y_test_standard - Y_pred_MLP)**2))
    
    if  Cross_validation_Accuracy_MLP > Max_local:
        Max_local = Cross_validation_Accuracy_MLP
        Y_pred_MLP = best_model.predict(X_test_standard)
        Y_pred_MLP_train = best_model.predict(X_train_standard)

        Score_local = r2_score(Y_test_standard, Y_pred_MLP)
        Score_local_train = r2_score(Y_train_standard, Y_pred_MLP_train)
    
    MLP_scores['CV'].append(Max_local)
    MLP_scores['R2'].append(Score_local)
    MLP_scores['R2_train'].append(Score_local_train)
    
    if Max_local > MaxGlobal and Score_local > MaxGlobal_Score and Score_local_train > MaxGlobal_Score_train:
        MaxGlobal=Max_local
        MaxGlobal_Score= Score_local
        MaxGlobal_Score_train = Score_local_train
        MLP_BestFit = best_model
        MLP_BestFit_iter = (j)
        print(MLP_BestFit_iter)
        MLP_X_train_standard=X_train_standard
        MLP_Y_train_standard=Y_train_standard
        MLP_X_test_standard=X_test_standard
        MLP_Y_test_standard=Y_test_standard
        
        MLP_Y_train_standard_realscale=yscaler.inverse_transform(Y_train_standard)
        MLP_Y_test_standard_realscale=yscaler.inverse_transform(Y_test_standard)
        ypredtrain=MLP_BestFit.predict(X_train_standard)
        ypredtest=MLP_BestFit.predict(X_test_standard)
        MLP_pred_Y_train_standard_realscale=yscaler.inverse_transform(ypredtrain)
        MLP_pred_Y_test_standard_realscale=yscaler.inverse_transform(ypredtest)
   

In [ ]:
print(MLP_BestFit)

In [ ]:
S = pd.DataFrame(MLP_scores)

print("CV = ", S.CV)
print("R2 = ", S.R2)
print("R2 train = ", S.R2_train)

print("test r2 = {}".format(r2_score(MLP_Y_test_standard_realscale,MLP_pred_Y_test_standard_realscale)))
print("test RMSE: {0} and MAE: {1}".format(sqrt(np.mean((MLP_Y_test_standard_realscale - MLP_pred_Y_test_standard_realscale)**2)),mae(MLP_Y_test_standard_realscale, MLP_pred_Y_test_standard_realscale).round(3)))

print("train r2 = {}".format(r2_score(MLP_Y_train_standard_realscale,MLP_pred_Y_train_standard_realscale)))
print("train RMSE: {0} and MAE: {1}".format(sqrt(np.mean((MLP_Y_train_standard_realscale - MLP_pred_Y_train_standard_realscale)**2)),mae(MLP_Y_train_standard_realscale, MLP_pred_Y_train_standard_realscale).round(3)))
MLP_test_r2 = r2_score(MLP_Y_test_standard_realscale,MLP_pred_Y_test_standard_realscale)
MLP_test_RMSE = sqrt(np.mean((MLP_Y_test_standard_realscale - MLP_pred_Y_test_standard_realscale)**2))
MLP_train_r2 = r2_score(MLP_Y_train_standard_realscale, MLP_pred_Y_train_standard_realscale)
MLP_train_RMSE = sqrt(np.mean((MLP_Y_train_standard_realscale - MLP_pred_Y_train_standard_realscale)**2))

MLP_CV_Mean = S.CV.mean()
joblib.dump(MLP_BestFit, './model/MLP_model.pkl')
joblib.dump(xscaler, './model/MLP_Xscaler.pkl')
joblib.dump(yscaler, './model/MLP_Yscaler.pkl')
# Save data
np.save('./model_data/MLP_Y_train_standard_realscale',MLP_Y_train_standard_realscale)
np.save('./model_data/MLP_Y_test_standard_realscale',MLP_Y_test_standard_realscale)
np.save('./model_data/MLP_pred_Y_train_standard_realscale',MLP_pred_Y_train_standard_realscale)
np.save('./model_data/MLP_pred_Y_test_standard_realscale',MLP_pred_Y_test_standard_realscale)
np.save('./model_data/MLP_X_train_standard',MLP_X_train_standard)
np.save('./model_data/MLP_Y_train_standard',MLP_Y_train_standard)
np.save('./model_data/MLP_X_test_standard',MLP_X_test_standard)
np.save('./model_data/MLP_Y_test_standard',MLP_Y_test_standard)
np.save('./model_data/MLP_scores',MLP_scores)




In [ ]:
'''
Validation DataSets
'''
file_name = 'Test_Phase.xlsx'
dataset = pd.read_excel(file_name, header=0)
#print(dataset)
x1 = dataset['P(W)'].values
x2 = dataset['V(mm/s)'].values
x3 = dataset['h(um)'].values
# x4 = dataset['t(um)'].values
x5 = dataset['LED(J/mm)'].values
# x6 = dataset['SED(J/mm2)'].values
x7 = dataset['VED(J/mm3)'].values
x8 = dataset['Ni'].values
# x9 = dataset['Ti'].values
x10 = dataset['Dmin'].values
x11 = dataset['Dmax'].values
x12 = dataset['D10'].values
# x13 = dataset['D50'].values
x14 = dataset['D90'].values


y1 = dataset['Ms'].values
y2 = dataset['Mf'].values
y3 = dataset['As'].values
y4 = dataset['Af'].values
X = np.column_stack((x1.reshape(-1, 1), 
                     x2.reshape(-1, 1), 
                     x3.reshape(-1, 1),
                #      x4.reshape(-1, 1), 
                     x5.reshape(-1, 1),
                #      x6.reshape(-1, 1),
                     x7.reshape(-1, 1),
                     x8.reshape(-1, 1),
                #      x9.reshape(-1, 1),
                     x10.reshape(-1, 1),
                     x11.reshape(-1, 1),
                     x12.reshape(-1, 1),
                #      x13.reshape(-1, 1),
                     x14.reshape(-1, 1),
               
                     ))

Y = np.column_stack((y1,y2, y3, y4))

print(Y)

In [ ]:
'''
Validation DataSets
'''
x_standard = xscaler.transform(X)
y_standard = yscaler.transform(Y)
y_pre_standard = MLP_BestFit.predict(x_standard)
y_pre_real = yscaler.inverse_transform(y_pre_standard)
print(Y)
print(y_pre_real.round(0))

In [ ]:
'''
All DataSets
'''
file_name = 'NiTi_PhaseDatesets.xlsx'
dataset = pd.read_excel(file_name, header=0)
#print(dataset)
x1 = dataset['P(W)'].values
x2 = dataset['V(mm/s)'].values
x3 = dataset['h(um)'].values
# x4 = dataset['t(um)'].values
x5 = dataset['LED(J/mm)'].values
# x6 = dataset['SED(J/mm2)'].values
x7 = dataset['VED(J/mm3)'].values
x8 = dataset['Ni'].values
# x9 = dataset['Ti'].values
x10 = dataset['Dmin'].values
x11 = dataset['Dmax'].values
x12 = dataset['D10'].values
# x13 = dataset['D50'].values
x14 = dataset['D90'].values


y1 = dataset['Ms'].values
y2 = dataset['Mf'].values
y3 = dataset['As'].values
y4 = dataset['Af'].values
all_X = np.column_stack((x1.reshape(-1, 1), 
                     x2.reshape(-1, 1), 
                     x3.reshape(-1, 1),
                #      x4.reshape(-1, 1), 
                     x5.reshape(-1, 1),
                #      x6.reshape(-1, 1),
                     x7.reshape(-1, 1),
                     x8.reshape(-1, 1),
                #      x9.reshape(-1, 1),
                     x10.reshape(-1, 1),
                     x11.reshape(-1, 1),
                     x12.reshape(-1, 1),
                #      x13.reshape(-1, 1),
                     x14.reshape(-1, 1),
             
                     ))

all_Y = np.column_stack((y1,y2, y3, y4))


print(len(all_Y))

In [ ]:
'''
All DataSets
'''
all_x_standard = xscaler.transform(all_X)
all_y_standard = yscaler.transform(all_Y)
all_y_pre_standard = MLP_BestFit.predict(all_x_standard)
all_y_pre_real = yscaler.inverse_transform(all_y_pre_standard)
print(all_Y)
print(all_y_pre_real.round(0))

In [ ]:
'''
All DataSets
'''
all_y_pre_real = all_y_pre_real.round(0)
print("All DataSets r2 = {}".format(r2_score(all_Y, all_y_pre_real)))
print("All DataSets RMSE: {0} and MAE: {1}".format(sqrt(np.mean((all_Y - all_y_pre_real)**2)),mae(all_Y, all_y_pre_real).round(3)))  
